In [1]:
import sys,os,glob,copy
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator,interp2d
import matplotlib as mpl
from matplotlib.colors import LogNorm
from IPython.display import display, Markdown
from collections import OrderedDict
import pylhe
import glob
import pyslha
import xml.etree.ElementTree as ET

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

# Set output and input files

In [2]:
inputDir = '../MadAnalysis5/bin/ANALYSIS_0/'

### Get hepMC (input) folder and extract model info

In [3]:
inputFile = os.path.join(inputDir,'Input/_defaultset.list')
with open(inputFile,'r') as f:
    hepMCfile = f.read()
    hepDir = os.path.dirname(hepMCfile)
print('HepMC input dir=',hepDir)

HepMC input dir= /home/lessa/MonoXSMS


#### Get Model Parameters

In [5]:
try:
    banner = sorted(glob.glob(hepDir+'/*banner.txt'),key=os.path.getmtime,reverse=True)
    if len(banner) == 0:
        print('Banner not found for %s' %label)
    elif len(banner) > 1:        
        print('\n%i banner files found for %s. Using %s' 
              %(len(banner),label,os.path.basename(banner[0])))
    banner = banner[0]
    xtree = ET.parse(banner)
    xroot = xtree.getroot()
    genInfo = xroot.find('header').find('MGGenerationInfo').text.strip().split('\n')
    genInfo = [x.replace('#','').strip().split(':') for x in genInfo]
    nevts = [eval(x[1]) for x in genInfo if 'Number of Events' in x[0]][0]
    xsecPBall = [eval(x[1]) for x in genInfo if 'Integrated weight (pb)' in x[0]]
    xsecPBmatched = [eval(x[1]) for x in genInfo if 'Matched Integrated weight (pb)' in x[0]]
    if xsecPBmatched:
        nevts = nevts*(xsecPBmatched[0]/xsecPBall[0])
        xsecPB = xsecPBmatched[0]
    else:
        xsecPB = xsecPBall[0]

    slha = xroot.find('header').find('slha').text
    pars = pyslha.readSLHA(slha)
    mMed = pars.blocks['MASS'][55]
    mDM = pars.blocks['MASS'][52]
    gVq = pars.blocks['DMINPUTS'][4] # Mediator-quark vector coupling
    gAq = pars.blocks['DMINPUTS'][10] # Mediator-quark axial coupling
    gVx = pars.blocks['DMINPUTS'][2] # Mediator-DM vector coupling
    gAx = pars.blocks['DMINPUTS'][3] # Mediator-DM axial coupling
    print('Cross-section (pb) = %1.3e' %xsecPB)
    print('Number of Events = %i' %nevts)
    print('mMed = %1.2f GeV, mDM = %1.2f GeV, gVq = %1.2f, gAq = %1.2f, gVx = %1.2f, gAx = %1.2f' 
          %(mMed,mDM,gVq,gAq,gVx,gAx))
except:
    banner = None

In [6]:
dataDict = {}
if banner:
    dataDict['filename'] = hepMCfile
    dataDict['Total xsec (pb)'] = xsecPB
    dataDict['Total MC Events'] = nevts
    if gVx != 0:
        dataDict['Coupling'] = 'Vector'
    else:
        dataDict['Coupling'] = 'Axial'

    dataDict['Mode'] = 'DM+QCDjets'

    dataDict['$m_{med}$'] = mMed
    dataDict['$m_{DM}$'] = mDM
    if dataDict['Coupling'] == 'Vector':
        dataDict['$g_{DM}$'] = gVx
        dataDict['$g_{q}$'] = gVq
    else:
        dataDict['$g_{DM}$'] = gAx
        dataDict['$g_{q}$'] = gAq


### Get efficiencies

In [7]:
effFile = os.path.join(inputDir,'Output/SAF/CLs_output_summary.dat')
with open(effFile,'r') as f:
    for il,l in enumerate(f.readlines()):
        if il == 0:
            l = l.replace('#','')
            header = [x.strip() for x in l.split('  ') if x.strip()]
            dataDict.update({col : [] for col in header})
        else:
            pt = [None]*len(dataDict)
            if not l.split():
                continue
            for ix,x in enumerate(l.split()):
                if not x.strip():
                    continue
                try:
                    x = eval(x)
                except:
                    pass
                if ix > len(dataDict)-1:
                    continue
                pt[ix] = x
            for icol,col in enumerate(header):
                dataDict[col].append(pt[icol])
if '||' in dataDict:
    dataDict.pop('||')

In [8]:
df = pd.DataFrame.from_dict(dataDict)

In [10]:
df['$M_{D}$'] = 11
df['$d$'] = 3

In [11]:
df

,dataset name,analysis name,signal region,best?,sig95(exp),sig95(obs),1-CLs,efficiency,stat,$M_{D}$,$d$
0,defaultset,cms_exo_20_004,monojet_SR_2016_bin0,0,6.141688,7.299051,0.0044,0.00826,0.044980,11,3
1,defaultset,cms_exo_20_004,monojet_SR_2016_bin1,0,3.880840,4.377352,0.0072,0.00708,0.041668,11,3
2,defaultset,cms_exo_20_004,monojet_SR_2016_bin2,0,2.576033,2.700787,0.0178,0.00468,0.033918,11,3
3,defaultset,cms_exo_20_004,monojet_SR_2016_bin3,0,1.810842,1.570962,0.0339,0.00424,0.032292,11,3
4,defaultset,cms_exo_20_004,monojet_SR_2016_bin4,0,1.327743,1.620786,0.0209,0.00366,0.030010,11,3
...,...,...,...,...,...,...,...,...,...,...,...
62,defaultset,cms_exo_20_004,monojet_SR_2018_bin18,0,0.229760,0.442677,0.0206,0.00066,0.012763,11,3
63,defaultset,cms_exo_20_004,monojet_SR_2018_bin19,0,0.380324,0.516664,0.0565,0.00032,0.008889,11,3
64,defaultset,cms_exo_20_004,monojet_SR_2018_bin20,0,0.328985,0.487985,0.0537,0.00032,0.008889,11,3
65,defaultset,cms_exo_20_004,monojet_SR_2018_bin21,1,0.128460,0.245564,0.0455,0.00100,0.015708,11,3


### Save DataFrame to pickle file

In [12]:
outputFile = os.path.join(inputDir,'summary.pcl')
print('Saving to',outputFile)
df.to_pickle(outputFile)

Saving to ../MadAnalysis5/bin/ANALYSIS_0/summary.pcl
